***
10 random seeds: range(20, 30)
for data creation for each type of spammer

invoke pgem for 10 random seeds: range(10)

get pgem accuracy (+- std dev), wacc and tau

save in results/spammer_type/pgem.csv
***

## Device Setup

In [1]:
!nvidia-smi

Mon Dec 22 05:34:14 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.54.03              Driver Version: 535.54.03    CUDA Version: 12.5     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000000:17:00.0 Off |                    0 |
| N/A   73C    P0             300W / 300W |  54610MiB / 81920MiB |    100%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
import os
import torch
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [3]:
print(f"Current PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")

Current PyTorch version: 2.9.1+cu128
CUDA available: True
CUDA version: 12.8


### Importing PGEM

In [4]:
import sys
sys.path.insert(0, "../")
sys.path.insert(1, "../../")

from spammer_types import *
from util import *
import opt_fair
from distribution_utils import crowd_bt_dist, logistic_preference_dist, comparisons_to_df, safe_kendalltau, to_numpy
from metrics import compute_acc, compute_weighted_acc
from pgem import EMWrapper

## Passage dataset

### Get the original df of passage dataset

In [5]:
df_path = "../../real_data/faceage/data/crowd_labels.csv"

In [6]:
import pandas as pd
df = pd.read_csv(df_path)
def sort_df(df, column_name):
        # Sort by a specific column (replace 'column_name' with your column)
        df_sorted = df.sort_values(by=column_name, ascending=True)  # or ascending=False

        return df_sorted
df = sort_df(df, 'performer')
df[['left', 'right', 'label', 'performer']].head()

,left,right,label,performer
0,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6306,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6176,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6175,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0
6174,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,https://tlk.s3.yandex.net/annotation_tasks/IMD...,0


In [7]:
percents = [10, 20, 40, 60, 80]
# percents = [10]

In [8]:
import pickle

with open("../../real_data/faceage/data/FaceAgeDF.pickle", "rb") as handle:
    df_passage = pickle.load(handle)
df_passage

,full_path,score,gender
0,nm1442940_rm3965098752_1996-10-3_2006.jpg,10,0.0
1,nm4832920_rm1781768448_2003-8-28_2013.jpg,10,0.0
2,nm0652089_rm860657920_1992-3-10_2002.jpg,10,0.0
3,nm0004917_rm1493730304_1969-5-12_1979.jpg,10,0.0
4,nm1113550_rm1332711936_1996-4-14_2006.jpg,10,0.0
...,...,...,...
9145,475367_1941-08-03_2011.jpg,70,1.0
9146,304085_1919-07-07_1989.jpg,70,1.0
9147,nm0001627_rm4164078592_1927-2-20_1997.jpg,70,1.0
9148,nm0000024_rm1715129344_1904-4-14_1974.jpg,70,1.0


In [9]:
size = len(df_passage)
print(size)
classes = [0] * size
# for faceage it would be classes = df_passage['gender']

9150


In [10]:
gt_df = df_passage

### Addition of Random Guessors

In [11]:
spammer_type = "random"

In [12]:
csv_file = f"results/{spammer_type}/pgem.csv"

In [13]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [14]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "PGEM_acc_mean", "PGEM_acc_std",
    "PGEM_wacc_mean", "PGEM_wacc_std",
    "PGEM_tau_mean", "PGEM_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    PGEM_accs, PGEM_waccs, PGEM_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_random_spammer(df, percent, seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                pg = EMWrapper(PC_faceage, max_iter_pgem, device, random_seed=seed)
                r_est, beta_est, ll = pg.run_algorithm()
                r_est = to_numpy(r_est)
                if np.isnan(r_est).any() or np.isnan(beta_est).any() or np.isnan(ll):
                    continue
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if PGEM_tau < 0:
                    r_est = -r_est
                PGEM_acc = compute_acc(gt_df, r_est, device)
                PGEM_wacc = compute_weighted_acc(gt_df, r_est, device)
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"PGEM failed due to {e}")
                continue
            PGEM_accs.append(PGEM_acc)
            PGEM_waccs.append(PGEM_wacc)
            PGEM_taus.append(PGEM_tau)
    
    row = [
        percent,
        np.mean(PGEM_accs), np.std(PGEM_accs),
        np.mean(PGEM_waccs), np.std(PGEM_waccs),
        np.mean(PGEM_taus), np.std(PGEM_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"PGEM | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(PGEM_accs):.4f} ± {np.std(PGEM_accs):.4f} | "
    f"WAcc: {np.mean(PGEM_waccs):.4f} ± {np.std(PGEM_waccs):.4f} | "
    f"Tau: {np.mean(PGEM_taus):.4f} ± {np.std(PGEM_taus):.4f}")

Unique performers: 4500
4500
cuda
cuda


  2%|██▍                                                                                                                    | 2/100 [00:00<00:17,  5.51it/s]

Iter 000: Log-likelihood = -0.387098


 24%|████████████████████████████▎                                                                                         | 24/100 [00:04<00:13,  5.64it/s]


Converged at iter 24, Log-likelihood change = 8.642673e-07
cuda
cuda


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:55<00:00,  1.79it/s]


cuda
cuda


  1%|█▏                                                                                                                     | 1/100 [00:00<01:02,  1.59it/s]

Iter 000: Log-likelihood = -0.450929


  7%|████████▎                                                                                                              | 7/100 [00:08<01:56,  1.25s/it]

### Addition of Anti-Personas

In [ ]:
spammer_type = "anti"

In [ ]:
csv_file = f"results/{spammer_type}/pgem.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "PGEM_acc_mean", "PGEM_acc_std",
    "PGEM_wacc_mean", "PGEM_wacc_std",
    "PGEM_tau_mean", "PGEM_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    PGEM_accs, PGEM_waccs, PGEM_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_anti_personas(df, percent, seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                pg = EMWrapper(PC_faceage, max_iter_pgem, device, random_seed=seed)
                r_est, beta_est, ll = pg.run_algorithm()
                r_est = to_numpy(r_est)
                if np.isnan(r_est).any() or np.isnan(beta_est).any() or np.isnan(ll):
                    continue
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if PGEM_tau < 0:
                    r_est = -r_est
                PGEM_acc = compute_acc(gt_df, r_est, device)
                PGEM_wacc = compute_weighted_acc(gt_df, r_est, device)
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"PGEM failed due to {e}")
                continue
            PGEM_accs.append(PGEM_acc)
            PGEM_waccs.append(PGEM_wacc)
            PGEM_taus.append(PGEM_tau)
    
    row = [
        percent,
        np.mean(PGEM_accs), np.std(PGEM_accs),
        np.mean(PGEM_waccs), np.std(PGEM_waccs),
        np.mean(PGEM_taus), np.std(PGEM_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"PGEM | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(PGEM_accs):.4f} ± {np.std(PGEM_accs):.4f} | "
    f"WAcc: {np.mean(PGEM_waccs):.4f} ± {np.std(PGEM_waccs):.4f} | "
    f"Tau: {np.mean(PGEM_taus):.4f} ± {np.std(PGEM_taus):.4f}")

### Addition of Left Position Biased Spammers

In [ ]:
spammer_type = "left"

In [ ]:
csv_file = f"results/{spammer_type}/pgem.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "PGEM_acc_mean", "PGEM_acc_std",
    "PGEM_wacc_mean", "PGEM_wacc_std",
    "PGEM_tau_mean", "PGEM_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    PGEM_accs, PGEM_waccs, PGEM_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_position_biased_spammers(df, percent,position_bias="left", seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                pg = EMWrapper(PC_faceage, max_iter_pgem, device, random_seed=seed)
                r_est, beta_est, ll = pg.run_algorithm()
                r_est = to_numpy(r_est)
                if np.isnan(r_est).any() or np.isnan(beta_est).any() or np.isnan(ll):
                    continue
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if PGEM_tau < 0:
                    r_est = -r_est
                PGEM_acc = compute_acc(gt_df, r_est, device)
                PGEM_wacc = compute_weighted_acc(gt_df, r_est, device)
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"PGEM failed due to {e}")
                continue
            PGEM_accs.append(PGEM_acc)
            PGEM_waccs.append(PGEM_wacc)
            PGEM_taus.append(PGEM_tau)
    
    row = [
        percent,
        np.mean(PGEM_accs), np.std(PGEM_accs),
        np.mean(PGEM_waccs), np.std(PGEM_waccs),
        np.mean(PGEM_taus), np.std(PGEM_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"PGEM | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(PGEM_accs):.4f} ± {np.std(PGEM_accs):.4f} | "
    f"WAcc: {np.mean(PGEM_waccs):.4f} ± {np.std(PGEM_waccs):.4f} | "
    f"Tau: {np.mean(PGEM_taus):.4f} ± {np.std(PGEM_taus):.4f}")

### Addition of Right Position Biased Spammers

In [11]:
spammer_type = "right"

In [12]:
csv_file = f"results/{spammer_type}/pgem.csv"

In [13]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [14]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "PGEM_acc_mean", "PGEM_acc_std",
    "PGEM_wacc_mean", "PGEM_wacc_std",
    "PGEM_tau_mean", "PGEM_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    PGEM_accs, PGEM_waccs, PGEM_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_position_biased_spammers(df, percent,position_bias="right", seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                pg = EMWrapper(PC_faceage, max_iter_pgem, device, random_seed=seed)
                r_est, beta_est, ll = pg.run_algorithm()
                r_est = to_numpy(r_est)
                if np.isnan(r_est).any() or np.isnan(beta_est).any() or np.isnan(ll):
                    continue
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if PGEM_tau < 0:
                    r_est = -r_est
                PGEM_acc = compute_acc(gt_df, r_est, device)
                PGEM_wacc = compute_weighted_acc(gt_df, r_est, device)
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"PGEM failed due to {e}")
                continue
            PGEM_accs.append(PGEM_acc)
            PGEM_waccs.append(PGEM_wacc)
            PGEM_taus.append(PGEM_tau)
    
    row = [
        percent,
        np.mean(PGEM_accs), np.std(PGEM_accs),
        np.mean(PGEM_waccs), np.std(PGEM_waccs),
        np.mean(PGEM_taus), np.std(PGEM_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"PGEM | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(PGEM_accs):.4f} ± {np.std(PGEM_accs):.4f} | "
    f"WAcc: {np.mean(PGEM_waccs):.4f} ± {np.std(PGEM_waccs):.4f} | "
    f"Tau: {np.mean(PGEM_taus):.4f} ± {np.std(PGEM_taus):.4f}")

Unique performers: 4500
4500
cuda
cuda


  2%|██▍                                                                                                                    | 2/100 [00:00<00:19,  5.03it/s]

Iter 000: Log-likelihood = -0.387161


 54%|███████████████████████████████████████████████████████████████▋                                                      | 54/100 [00:08<00:07,  6.04it/s]


Converged at iter 54, Log-likelihood change = 6.854534e-07
cuda
cuda


...

Converged at iter 17, Log-likelihood change = 7.450581e-07
cuda


cuda


  0%|                                                                                                                               | 0/100 [00:00<?, ?it/s]

Iter 000: Log-likelihood = -0.480086


 37%|███████████████████████████████████████████▋                                                                          | 37/100 [00:07<00:07,  8.99it/s]

### Addition of Equal Proportion of Spammers

In [ ]:
spammer_type = "equal"

In [ ]:
csv_file = f"results/{spammer_type}/pgem.csv"

In [ ]:
import os
os.makedirs(f"results/{spammer_type}", exist_ok=True)

In [ ]:
import csv
# -------------------------
# Write CSV header
# -------------------------
header = [
    "percent",
    "PGEM_acc_mean", "PGEM_acc_std",
    "PGEM_wacc_mean", "PGEM_wacc_std",
    "PGEM_tau_mean", "PGEM_tau_std"
]

with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(header)

In [ ]:
max_iter_pgem = 100

for percent in percents:
    # initialize metrics
    PGEM_accs, PGEM_waccs, PGEM_taus = [], [], []
    
    for sd in range(20, 30):
        
        # get df
        random_df, spammer_ids = add_equal_proportion_of_all_spammers(df, percent, seed=sd)
        PC_faceage = df_to_pickle_faceage(random_df, df_passage)
        K = len(PC_faceage.keys())
        print(K)
        
        for seed in range(10):
            try:
                pg = EMWrapper(PC_faceage, max_iter_pgem, device, random_seed=seed)
                r_est, beta_est, ll = pg.run_algorithm()
                r_est = to_numpy(r_est)
                if np.isnan(r_est).any() or np.isnan(beta_est).any() or np.isnan(ll):
                    continue
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
                if PGEM_tau < 0:
                    r_est = -r_est
                PGEM_acc = compute_acc(gt_df, r_est, device)
                PGEM_wacc = compute_weighted_acc(gt_df, r_est, device)
                PGEM_tau = safe_kendalltau(r_est, gt_df['score'].to_numpy())
            
            except Exception as e:
                print(f"PGEM failed due to {e}")
                continue
            PGEM_accs.append(PGEM_acc)
            PGEM_waccs.append(PGEM_wacc)
            PGEM_taus.append(PGEM_tau)
    
    row = [
        percent,
        np.mean(PGEM_accs), np.std(PGEM_accs),
        np.mean(PGEM_waccs), np.std(PGEM_waccs),
        np.mean(PGEM_taus), np.std(PGEM_taus)
    ]
    
    with open(csv_file, mode='a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(row)
    print(
    f"PGEM | "
    f"Percent: {percent} |"
    f"Acc: {np.mean(PGEM_accs):.4f} ± {np.std(PGEM_accs):.4f} | "
    f"WAcc: {np.mean(PGEM_waccs):.4f} ± {np.std(PGEM_waccs):.4f} | "
    f"Tau: {np.mean(PGEM_taus):.4f} ± {np.std(PGEM_taus):.4f}")